In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [ ]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 1192, done.
remote: Counting objects: 100% (453/453), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 1192 (delta 284), reused 265 (delta 207), pack-reused 739
Receiving objects: 100% (1192/1192), 2.33 MiB | 4.77 MiB/s, done.
Resolving deltas: 100% (689/689), done.
/content/easy-few-shot-learning/easy-few-shot-learning
Processing /content/easy-few-shot-learning/easy-few-shot-learning
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for easyfsl: filename=easyfsl-1.5.0-py3-none-any.whl size=72953 sha256=dcadbe399291b7eac2bab3cb29ade719b61a23b11ab9fd11abd0b61be97b3d9e
  Stored in directory: /root/.cache/pip/wheels/1d/e1/c6/4ec7c2c4109013181c4f772ff34646b8b9eeb1a08887be35ed
Successfully built easyfsl
  Attempting uninstall: easyfsl
    Found existing installation: easyfsl 1.5.0
    Uninstalling eas

In [ ]:
cd ..


/content


In [ ]:
!ls

drive  easy-few-shot-learning  sample_data


In [ ]:

# Install gdown if not already installed
!pip install gdown

import gdown
import zipfile
import os

# https://drive.google.com/file/d/1d2xcqUN3WZI0MuPpxtEkBaRLQ2hGCGid/view?usp=sharing
# Google Drive file ID
file_id = "1DRlg6rf5oZWYElsJkonwkPxaDzfg9bGN"
destination = "/content/numta-c-processed.zip"

# Construct the download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(download_url, destination, quiet=False)

# Unzip the file
unzip_dir = "/content/data"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1DRlg6rf5oZWYElsJkonwkPxaDzfg9bGN
From (redirected): https://drive.google.com/uc?id=1DRlg6rf5oZWYElsJkonwkPxaDzfg9bGN&confirm=t&uuid=74ea5f59-87a8-412b-b7ec-9cf36ba65d5d
To: /content/numta-c-processed.zip
100%|██████████| 163M/163M [00:04<00:00, 38.4MB/s]


## Download or Upload the Dataset

In [ ]:

# Install gdown if not already installed
!pip install gdown

import gdown
import zipfile
import os

# https://drive.google.com/file/d/1d2xcqUN3WZI0MuPpxtEkBaRLQ2hGCGid/view?usp=sharing
# Google Drive file ID
file_id = "1uAzL2aFqfmn_7ysscd0PIEhtzMvX-pAb"
destination = "/content/hindi.zip"


# Construct the download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(download_url, destination, quiet=False)

# Unzip the file
unzip_dir = "/content/data"
os.makedirs(unzip_dir, exist_ok=True)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1uAzL2aFqfmn_7ysscd0PIEhtzMvX-pAb
From (redirected): https://drive.google.com/uc?id=1uAzL2aFqfmn_7ysscd0PIEhtzMvX-pAb&confirm=t&uuid=0d7b9d5e-89db-4cb7-89d2-3db455c0b4b6
To: /content/hindi.zip
100%|██████████| 79.8M/79.8M [00:00<00:00, 128MB/s]


In [ ]:
!unzip /content/BasicFinalDatabase.zip -d /content/data


Streaming output truncated to the last 5000 lines.
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000060.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000061.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000062.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000063.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000064.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000065.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000066.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000067.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000068.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000069.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000070.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000071.bmp  
  inflating: /content/data/BasicFinalDatabase/Train/201/bcc000072.bmp  
  inflating: 

In [ ]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import torchvision
import torch.utils.data

In [ ]:
random_seed = 30
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Episodic Training

In [ ]:
n_way = 3
n_shot = 1
n_query = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_workers = 0

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class BengaliCharactersDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                if img_file.endswith('.png'):
                    self.samples.append((os.path.join(class_dir, img_file), class_name))

        # Debug: Print the first few samples
        print("First few samples:")
        print(self.samples[:5])  # Adjust the number to print more or fewer samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_str = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Convert label from string to integer
        label = self.class_to_idx[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label

    def get_labels(self):
        return [label for _, label in self.samples]

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class BengaliCharactersDataset2(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                if img_file.endswith('.bmp'):
                    self.samples.append((os.path.join(class_dir, img_file), class_name))

        # Debug: Print the first few samples
        print("First few samples:")
        print(self.samples[:5])  # Adjust the number to print more or fewer samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_str = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Convert label from string to integer
        label = self.class_to_idx[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label

    def get_labels(self):
        return [label for _, label in self.samples]

In [ ]:
image_size = 84  # Adjusted for ResNet, which typically expects larger input sizes

train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Random rotation between -10 and 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize to [-1, 1]
])

test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize to [-1, 1]
])

train_set = BengaliCharactersDataset(root_dir='/content/data/DevanagariHandwrittenCharacterDataset/Train', transform=train_transforms)
test_set = BengaliCharactersDataset2(root_dir='/content/data/BasicFinalDatabase/Test', transform=test_transforms)

First few samples:
[('/content/data/DevanagariHandwrittenCharacterDataset/Train/digit_6/67868.png', 'digit_6'), ('/content/data/DevanagariHandwrittenCharacterDataset/Train/digit_6/41546.png', 'digit_6'), ('/content/data/DevanagariHandwrittenCharacterDataset/Train/digit_6/10102.png', 'digit_6'), ('/content/data/DevanagariHandwrittenCharacterDataset/Train/digit_6/53208.png', 'digit_6'), ('/content/data/DevanagariHandwrittenCharacterDataset/Train/digit_6/99269.png', 'digit_6')]
First few samples:
[('/content/data/BasicFinalDatabase/Test/186/bcc000036.bmp', '186'), ('/content/data/BasicFinalDatabase/Test/186/bcc000021.bmp', '186'), ('/content/data/BasicFinalDatabase/Test/186/bcc000050.bmp', '186'), ('/content/data/BasicFinalDatabase/Test/186/bcc000034.bmp', '186'), ('/content/data/BasicFinalDatabase/Test/186/bcc000001.bmp', '186')]


In [ ]:
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 500
n_validation_tasks = 100

val_set = test_set

train_set.get_labels = lambda: [
    instance[1] for instance in train_set
]

val_set.get_labels = lambda: [
    instance[1] for instance in val_set
]

# Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

# Finally, the DataLoader. We customize the collate_fn so that batches are delivered
# in the shape: (support_images, support_labels, query_images, query_labels, class_ids)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

print(len(train_set), len(val_set))

78200 3000


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier
# Define the CNN Encoder
class CNNEncoder(nn.Module):
    """CNN Encoder for feature extraction."""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=0),
            nn.BatchNorm2d(64, momentum=1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=0),
            nn.BatchNorm2d(64, momentum=1, affine=True),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64, momentum=1, affine=True),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64, momentum=1, affine=True),
            nn.ReLU()
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)  # Flatten the output tensor
        return out  # Return the 1-dimensional tensor for feature embedding

# Load a pretrained ResNet18 model and modify it to output embeddings
class ResNetEncoder(nn.Module):
    def __init__(self, base_model):
        super(ResNetEncoder, self).__init__()
        self.features = nn.Sequential(*list(base_model.children())[:-1])  # Remove the final classification layer

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return x

# Instantiate the CNN Encoder and the ResNet Encoder
cnn_encoder = CNNEncoder()
pretrained_resnet18 = models.resnet18(pretrained=True)
resnet_encoder = ResNetEncoder(pretrained_resnet18)

# Combine the features from both encoders
class CombinedEncoder(nn.Module):
    def __init__(self, cnn_encoder, resnet_encoder):
        super(CombinedEncoder, self).__init__()
        self.cnn_encoder = cnn_encoder
        self.resnet_encoder = resnet_encoder

    def forward(self, x):
        cnn_features = self.cnn_encoder(x)
        resnet_features = self.resnet_encoder(x)
        combined_features = torch.cat((cnn_features, resnet_features), dim=1)
        return combined_features

# Instantiate the Combined Encoder
combined_encoder = CombinedEncoder(cnn_encoder, resnet_encoder)

# Ensure the model is set to the correct device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
combined_encoder = combined_encoder.to(DEVICE)

# Create the few-shot classifier with the Combined Encoder
few_shot_classifier = PrototypicalNetworks(combined_encoder).to(DEVICE)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 30
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [ ]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))

            loss.backward()

            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [ ]:
from easyfsl.utils import evaluate


best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = few_shot_classifier.state_dict()
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

    if epoch % 1 == 0 : torch.save(best_state, '/content/1shot_5way_hindi_BFD.pth')

Epoch 0


Validation: 100%|██████████| 100/100 [00:06<00:00, 14.68it/s, accuracy=0.535]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|██████████| 100/100 [00:06<00:00, 16.44it/s, accuracy=0.572]


Ding ding ding! We found a new best model!
Epoch 2


...

Validation: 100%|██████████| 100/100 [00:06<00:00, 15.53it/s, accuracy=0.53]


Epoch 28


Validation: 100%|██████████| 100/100 [00:06<00:00, 14.34it/s, accuracy=0.497]


Epoch 29


Validation: 100%|██████████| 100/100 [00:06<00:00, 16.04it/s, accuracy=0.504]


In [ ]:
# PATH = '/content/drive/MyDrive/PatternRecognition-main/ML/Proto/5shot_5way_BanglaPrototypicalNetworks_BanglaLekha_Isolated.pth'
# few_shot_classifier.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))


In [ ]:
PATH = '/content/1shot_5way_hindi_BFD.pth'
few_shot_classifier.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## Code for combininng the Train and Test folders

In [ ]:
# import os
# import shutil
# train_dir = '/content/data/training-c-processed/Train'
# test_dir = '/content/data/training-c-processed/Test'
# new_test_dir = '/content/data/training-c-processed/New_Test'

# os.makedirs(new_test_dir, exist_ok=True)

# def copy_files(src_dir, dest_dir):
#     for class_name in os.listdir(src_dir):
#         class_path = os.path.join(src_dir, class_name)
#         if os.path.isdir(class_path):
#             dest_class_path = os.path.join(dest_dir, class_name)
#             os.makedirs(dest_class_path, exist_ok=True)
#             for img_file in os.listdir(class_path):
#                 src_img_path = os.path.join(class_path, img_file)
#                 dest_img_path = os.path.join(dest_class_path, img_file)
#                 shutil.copy(src_img_path, dest_img_path)

# copy_files(train_dir, new_test_dir)
# copy_files(test_dir, new_test_dir)


**Evaluation**

In [ ]:
############################## Test on digits 6-9 #####################################

n_test_tasks = 100
test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

############################## Test on digits 0-9 #####################################

# new_test_set = BengaliCharactersDataset(root_dir='/content/data/training-b-processed/New_Test', transform=test_transforms)
# n_test_tasks = 100

# new_test_set.get_labels = lambda: [
#     instance[1] for instance in new_test_set
# ]

# test_sampler = TaskSampler(
#     new_test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
# )
# test_loader = DataLoader(
#     new_test_set,
#     batch_sampler=test_sampler,
#     num_workers=n_workers,
#     pin_memory=True,
#     collate_fn=test_sampler.episodic_collate_fn,
# )

In [ ]:
from easyfsl.utils import evaluate
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 100/100 [00:07<00:00, 12.91it/s, accuracy=0.506]

Average accuracy : 50.62 %


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import torch


def evaluate_with_metrics(model, loader, device):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for _, (support_images, support_labels, query_images, query_labels, _) in enumerate(loader):
            model.process_support_set(support_images.to(device), support_labels.to(device))
            outputs = model(query_images.to(device))
            _, preds = torch.max(outputs, 1)
            all_labels.extend(query_labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    return precision, recall, f1_score


In [ ]:
precision, recall, f1_score = evaluate_with_metrics(few_shot_classifier, test_loader, DEVICE)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Precision: 0.50
Recall: 0.50
F1 Score: 0.50


## Code for individul prediction

In [ ]:
def predict_image_with_support_set(query_image_path):
    # Load and transform the query image
    query_image = Image.open(query_image_path).convert('RGB')
    query_image_tensor = image_transforms(query_image).unsqueeze(0)  # Add batch dimension

    # Set the model in evaluation mode
    few_shot_classifier.eval()

    # Use the model to predict
    with torch.no_grad():
        few_shot_classifier.process_support_set(support_images_tensor.to(DEVICE), support_labels_tensor.to(DEVICE))
        outputs = few_shot_classifier(query_image_tensor.to(DEVICE))

    probabilities = torch.softmax(outputs, dim=1)
    predicted_label_idx = probabilities.argmax(1).item()
    predicted_label = support_labels[predicted_label_idx] + 1  # Adjust label to match original range

    return predicted_label



import torch
from torchvision import transforms
from PIL import Image
import os


label_to_bengali = {
    1: "অ",
    2: "আ",
    3: "ই",
    4: "ঈ",
    5: "উ",
    6: "ঊ",
    7: "ঋ",
    8: "এ",
    9: "ঐ",
    10: "ও",
    11: "ঔ",
    12: "ক",
    13: "খ",
    14: "গ",
    15: "ঘ",
    16: "ঙ",
    17: "চ",
    18: "ছ",
    19: "জ",
    20: "ঝ",
    21: "ঞ",
    22: "ট",
    23: "ঠ",
    24: "ড",
    25: "ঢ",
    26: "ণ",
    27: "ত",
    28: "থ",
    29: "দ",
    30: "ধ",
    31: "ন",
    32: "প",
    33: "ফ",
    34: "ব",
    35: "ভ",
    36: "ম",
    37: "য",
    38: "র",
    39: "ল",
    40: "শ",
    41: "ষ",
    42: "স",
    43: "হ",
    44: "ড়",
    45: "ঢ়",
    46: "য়",
    47: "ৎ",
    48: "ং",
    49: "ঃ",
    50: "ঁ",
    51: "০",
    52: "১",
    53: "২",
    54: "৩",
    55: "৪",
    56: "৫",
    57: "৬",
    58: "৭",
    59: "৮",
    60: "৯",
}


# Function to print the Bengali character for a given label
def print_bengali_character(label):
    character = label_to_bengali.get(label, "Unknown label")
    print(character)


# Define the image preprocessing
image_transforms = transforms.Compose([
    transforms.Resize((84, 84)),  # Resize the image to what the model expects
    transforms.ToTensor(),        # Convert the image to a tensor
])

# Define the root directory of your support images
root_dir = '/content/data/BanglaLekha_Isolated_mod/Test/'

# Generate support set
support_images = []
support_labels = []

# Assuming each class directory contains images for that class
for label in range(1, 61):  # Labels from 1 to 60
    class_dir = os.path.join(root_dir, str(label))
    if not os.path.exists(class_dir):
        continue  # Skip if the class directory doesn't exist

    # List all images in the class directory
    img_files = [f for f in os.listdir(class_dir) if f.endswith('.png')]
    for img_file in img_files[:1]:  # Take only the first image for simplicity; adjust as needed
        img_path = os.path.join(class_dir, img_file)
        image = Image.open(img_path).convert('RGB')
        support_images.append(image_transforms(image))
        support_labels.append(label - 1)  # Adjust label to start from 0

# Convert lists to tensors
support_images_tensor = torch.stack(support_images)
support_labels_tensor = torch.tensor(support_labels)

query_image_path = '/content/mota_dho.jpg'
predicted_label = predict_image_with_support_set(query_image_path)
print("Predicted label: ")
print_bengali_character(predicted_label)

In [ ]:
# !pip install Pillow

# import torch
# import torch.nn as nn
# import torchvision.models as models
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt
# import numpy as np
# from PIL import Image
# # from easyfsl.methods import PrototypicalNetworks, FewShotClassifier

# # Define the CNN Encoder
# class CNNEncoder(nn.Module):
#     """CNN Encoder for feature extraction."""
#     def __init__(self):
#         super(CNNEncoder, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=3, padding=0),
#             nn.BatchNorm2d(64, momentum=1, affine=True),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, padding=0),
#             nn.BatchNorm2d(64, momentum=1, affine=True),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )
#         self.layer3 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64, momentum=1, affine=True),
#             nn.ReLU()
#         )
#         self.layer4 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64, momentum=1, affine=True),
#             nn.ReLU()
#         )

#     def forward(self, x):
#         out = self.layer1(x)
#         out1 = out.clone()  # Save output for visualization
#         print(f"Layer 1 output shape: {out1.shape}")
#         out = self.layer2(out)
#         out2 = out.clone()  # Save output for visualization
#         print(f"Layer 2 output shape: {out2.shape}")
#         out = self.layer3(out)
#         out3 = out.clone()  # Save output for visualization
#         print(f"Layer 3 output shape: {out3.shape}")
#         out = self.layer4(out)
#         out4 = out.clone()  # Save output for visualization
#         print(f"Layer 4 output shape: {out4.shape}")
#         out = out.view(out.size(0), -1)  # Flatten the output tensor
#         print(f"Flattened output shape: {out.shape}")
#         return out, [out1, out2, out3, out4]  # Return the 1-dimensional tensor and intermediate features

# # Load a pretrained ResNet18 model and modify it to output embeddings
# class ResNetEncoder(nn.Module):
#     def __init__(self, base_model):
#         super(ResNetEncoder, self).__init__()
#         self.base_model = base_model
#         self.intermediate_outputs = []

#         # Extract intermediate layers
#         self.layer1 = nn.Sequential(*list(self.base_model.children())[:4])
#         self.layer2 = nn.Sequential(*list(self.base_model.children())[4:5])
#         self.layer3 = nn.Sequential(*list(self.base_model.children())[5:6])
#         self.layer4 = nn.Sequential(*list(self.base_model.children())[6:7])
#         self.layer5 = nn.Sequential(*list(self.base_model.children())[7:8])

#     def forward(self, x):
#         out = self.layer1(x)
#         out1 = out.clone()  # Save output for visualization
#         print(f"ResNet Layer 1 output shape: {out1.shape}")
#         out = self.layer2(out)
#         out2 = out.clone()  # Save output for visualization
#         print(f"ResNet Layer 2 output shape: {out2.shape}")
#         out = self.layer3(out)
#         out3 = out.clone()  # Save output for visualization
#         print(f"ResNet Layer 3 output shape: {out3.shape}")
#         out = self.layer4(out)
#         out4 = out.clone()  # Save output for visualization
#         print(f"ResNet Layer 4 output shape: {out4.shape}")
#         out = self.layer5(out)
#         out5 = out.clone()  # Save output for visualization
#         print(f"ResNet Layer 5 output shape: {out5.shape}")
#         out = torch.flatten(out, 1)
#         print(f"Flattened ResNet output shape: {out.shape}")
#         return out, [out1, out2, out3, out4, out5]  # Return the 1-dimensional tensor and intermediate features

# # Instantiate the CNN Encoder and the ResNet Encoder
# cnn_encoder = CNNEncoder()
# pretrained_resnet18 = models.resnet18(pretrained=True)
# resnet_encoder = ResNetEncoder(pretrained_resnet18)

# # Combine the features from both encoders
# class CombinedEncoder(nn.Module):
#     def __init__(self, cnn_encoder, resnet_encoder):
#         super(CombinedEncoder, self).__init__()
#         self.cnn_encoder = cnn_encoder
#         self.resnet_encoder = resnet_encoder

#     def forward(self, x):
#         cnn_features, cnn_intermediate_features = self.cnn_encoder(x)
#         resnet_features, resnet_intermediate_features = self.resnet_encoder(x)
#         combined_features = torch.cat((cnn_features, resnet_features), dim=1)
#         print(f"Combined features shape: {combined_features.shape}")
#         return combined_features, cnn_intermediate_features, resnet_intermediate_features

# # Instantiate the Combined Encoder
# combined_encoder = CombinedEncoder(cnn_encoder, resnet_encoder)

# # Ensure the model is set to the correct device
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# combined_encoder = combined_encoder.to(DEVICE)

# # Create the few-shot classifier with the Combined Encoder
# # few_shot_classifier = PrototypicalNetworks(combined_encoder).to(DEVICE)

# # # Function to visualize feature maps
# # def visualize_feature_maps(feature_maps, layer_num, encoder_name):
# #     feature_maps = feature_maps.cpu().detach().numpy()
# #     num_maps = feature_maps.shape[1]
# #     size = feature_maps.shape[2]
# #     grid_size = int(np.ceil(np.sqrt(num_maps)))

# #     fig, axes = plt.subplots(grid_size, grid_size, figsize=(12, 12))
# #     fig.suptitle(f"{encoder_name} Layer {layer_num} Feature Maps")

# #     for i in range(grid_size):
# #         for j in range(grid_size):
# #             if i * grid_size + j < num_maps:
# #                 ax = axes[i, j]
# #                 ax.imshow(feature_maps[0, i * grid_size + j, :, :], cmap='gray')
# #                 ax.axis('off')
# #             else:
# #                 axes[i, j].axis('off')

# #     plt.show()

# # # Load and preprocess the image
# # def load_image(image_path):
# #     image = Image.open(image_path).convert('RGB')
# #     preprocess = transforms.Compose([
# #         transforms.Resize((84, 84)),
# #         transforms.ToTensor(),
# #         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# #     ])
# #     image = preprocess(image).unsqueeze(0)  # Add batch dimension
# #     return image.to(DEVICE)

# # # Example usage to visualize features:
# # image_path = '/content/cat.jpg'  # Path to your image file
# # input_image = load_image(image_path)
# # _, cnn_intermediate_features, resnet_intermediate_features = combined_encoder(input_image)

# # # Visualize feature maps for each CNN layer
# # for i, feature_map in enumerate(cnn_intermediate_features):
# #     visualize_feature_maps(feature_map, i + 1, 'CNN')

# # # Visualize feature maps for each ResNet layer
# # for i, feature_map in enumerate(resnet_intermediate_features):
# #     visualize_feature_maps(feature_map, i + 1, 'ResNet')


# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# import torch
# from torchvision import transforms
# import os

# # Function to visualize and save a single feature map
# def visualize_and_save_feature_map(feature_map, layer_num, encoder_name, save_dir):
#     feature_map = feature_map.cpu().detach().numpy()
#     fig, ax = plt.subplots(figsize=(6, 6))
#     # fig.suptitle(f"{encoder_name} Layer {layer_num} Feature Map")

#     # Visualize the first feature map
#     ax.imshow(feature_map[0, 0, :, :], cmap='gray')
#     ax.axis('off')

#     # Save the figure
#     save_path = os.path.join(save_dir, f"{encoder_name}_Layer_{layer_num}.png")
#     plt.savefig(save_path)
#     plt.close(fig)

# # Load and preprocess the image
# def load_image(image_path):
#     image = Image.open(image_path).convert('RGB')
#     preprocess = transforms.Compose([
#         transforms.Resize((84, 84)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])
#     image = preprocess(image).unsqueeze(0)  # Add batch dimension
#     return image.to(DEVICE)

# # Example usage to visualize and save features:
# image_path = '/content/ri.jpg'  # Path to your image file
# input_image = load_image(image_path)
# _, cnn_intermediate_features, resnet_intermediate_features = combined_encoder(input_image)

# # Create directory to save images
# save_dir = './feature_maps'
# os.makedirs(save_dir, exist_ok=True)

# # Visualize and save a single feature map for each CNN layer
# for i, feature_map in enumerate(cnn_intermediate_features):
#     visualize_and_save_feature_map(feature_map, i + 1, 'CNN', save_dir)

# # Visualize and save a single feature map for each ResNet layer
# for i, feature_map in enumerate(resnet_intermediate_features):
#     visualize_and_save_feature_map(feature_map, i + 1, 'ResNet', save_dir)
# prompt: zip a folder in collab

# !zip -r /content/feature_maps.zip /content/feature_maps

Layer 1 output shape: torch.Size([1, 64, 41, 41])
Layer 2 output shape: torch.Size([1, 64, 19, 19])
Layer 3 output shape: torch.Size([1, 64, 19, 19])
Layer 4 output shape: torch.Size([1, 64, 19, 19])
Flattened output shape: torch.Size([1, 23104])
ResNet Layer 1 output shape: torch.Size([1, 64, 21, 21])
ResNet Layer 2 output shape: torch.Size([1, 64, 21, 21])
ResNet Layer 3 output shape: torch.Size([1, 128, 11, 11])
ResNet Layer 4 output shape: torch.Size([1, 256, 6, 6])
ResNet Layer 5 output shape: torch.Size([1, 512, 3, 3])
Flattened ResNet output shape: torch.Size([1, 4608])
Combined features shape: torch.Size([1, 27712])
